In [1]:
import os
from ekonlpy import Mecab
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from ekonlpy import Mecab

def select_MPB(target_date):
    start_date = "2013-05-09"
    def date_range(start, end):
        start = datetime.strptime(start, "%Y-%m-%d")
        end = datetime.strptime(end, "%Y-%m")
        before_one_day = end - relativedelta(days=30)
        dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((before_one_day-start).days+1)]
        return dates

    dates = date_range(start_date, target_date)

    path = "./MPB_crawling_dataset"
    file_list = []
    if os.path.isdir(path):
        text_list = os.listdir(path)
        for i in text_list:
            if i[-3:] == "txt":
                target_file = i[:-4]
                if target_file in dates:
                    file_list.append(i)
    MPB_file = file_list[-1]                
    f = open(f"{path}/{MPB_file}", 'r')
    file_text = f.readline()
    return file_text



In [2]:
target_date = "2015-07" #예측 날짜
file_text = select_MPB(target_date)
print("file_text 생성")

file_text 생성


In [3]:
import pandas as pd

In [6]:
doveish_df = pd.read_csv("./total_doveish.csv", encoding_errors='ignore')
hawkish_df = pd.read_csv("./total_hawkish.csv", encoding_errors='ignore')
doveish_df["label"] = "doveish"
hawkish_df["label"] = "hawkish"
word_df = pd.concat([doveish_df,hawkish_df],axis=0)
x_tain = word_df["words"]
y_tain = word_df["label"]

In [7]:
file_text

' 국내외 경제동향과 관련하여 일부 위원은 관련 부서에서 그간 소득여건이 개선될 것이라고 판단하여 왔는데, 이번 달에는 실질임금 상승 정체 등으로 가계소득 개선이 미약한 가운데 소비심리 회복이 지연되면서 향후 소비 증가세가 제약될 가능성이 있는 것으로 평가하고 있다고 언급한 후, 금년 들어 근로자 가구의 실질가처분소득 증가율이 전체 가구에 비해 낮아진 것이 추세적 흐름인지 물었으며, 이에 대해 관련 부서에서는 근로자 가구의 실질가처분소득 증가율이 전체 가구에 비해 낮아진 것은 지난해 11월 이후 임금 수준과 임금 상승률이 낮은 서비스업종을 중심으로 중장년층의 취업이 늘어나면서 근로자 가구의 명목임금 상승률이 낮아진데 주로 기인하고 있다고 답변하였음.이어서 동위원은 경제상황을 종합해보면 경제성장률은 지난 전망에서 크게 벗어나지 않겠으나, 물가는 지난 전망치를 하회할 것 으로 보인다고 평가한 후, 경제성장률(2013년 실적치 3% →2014년 전 망치 3.5%) 기준으로 보면 경제상황이 나쁘지 않음에도 여러 가지 문제들이 제기되고 있는 것은 고용부문에 문제가 있기 때문이라고 강조하고, 이와 같이 경제성장률만으로 경제상황을 판단하기 어려운 상황인 점을 고려하여 고용보험 통계 등 고용·임금 관련 통계를 확충·분석할 필요가 있다는 견해를 피력하였음. 다른 일부 위원은 경기흐름의 방향성이 뚜렷하지 않은 가운데 과거 경기순환기 에 비해진폭과 지속성이 현저히 축소되고 있다고 지적한 후, 일반적으로 경기 변동성 축소는 불확실성 감소 등을 통해 투자를 촉진하고 성장률을 높이는 효과가 있지만, 최근의 경기변동성 축소는 경제여건의 불확실성이 상존하는 가운데 잠재성장률이 추세적으로 하락하는 상황에서 발생함에 따라 향후 성장잠재력 제고를 위한 모멘텀 확보가 어려울 수 있다는 우려를 표명하였음.이어서 동위원은 「한국은행 지역경제보고서」(2014년 4/4분기, 2014년 11월 발간 )의 모니터링 결과-금년 10∼11월 중국내 경기가 완만한 개선 추세를 이어가고 있으나 회복 모멘텀은 강하

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from ekonlpy import Mecab
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy_score(y_test, y_pred)
# precision_score(y_test, y_pred)
# recall_score(y_test, y_pred)
# f1_score(y_test, y_pred)

#모델 훈련및 결과출력
# def Model_test(word_list,file_text):
    # 1. 데이터 준비
X_train = x_tain
y_train = y_tain
# 2. 특성 추출 (형태소 분석 및 n-그램)
mecab = Mecab()
tagger = mecab.pos
mecab_tokenize = lambda text: [token for token, pos in tagger(text)]
vectorizer = CountVectorizer(tokenizer=mecab_tokenize, ngram_range=(1, 5), min_df=15)
X_train_counts = vectorizer.fit_transform(X_train)
# 3. 나이브 베이즈 모델 학습
clf = MultinomialNB()
clf.fit(X_train_counts, y_train)
print(clf.score(X_train_counts, y_train))
#4. 새로운 입력 예측
X_test = [file_text]
X_test_counts = vectorizer.transform(X_test)
predicted_labels = clf.predict(X_test_counts)
print(predicted_labels)

/Users/gimseyeong/anaconda3/envs/nlp/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.930124810892587
['doveish']
